In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA


In [2]:
housing_data = pd.read_csv('train.csv')

In [3]:
housing_data.head()

,id,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator,Price
0,26813,964,Hooghly Chinsurah,3,0,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,7500000
1,10365,1075,Virar West,2,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,6200000
2,17863,2092,Manikonda,3,0,1,1,1,1,0,...,0,1,0,0,0,0,0,0,0,10400000
3,24337,1174,Devanahalli,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,18600000
4,12590,1380,Kharghar,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9500000


In [4]:
housing_data = housing_data.drop(columns=["Resale"]) # Nhà mua lại hay không không ảnh hưởng tới giá
housing_data = housing_data.drop(columns=["Location"])
housing_data = housing_data.drop(columns=["id"])

In [5]:
housing_data.head()

,Area,No. of Bedrooms,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator,Price
0,964,3,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,7500000
1,1075,2,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,6200000
2,2092,3,1,1,1,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,10400000
3,1174,2,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,18600000
4,1380,2,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9500000


In [6]:
def mix_feature(housing_data, train_data=True):
  # Kết hợp các biến số học
  housing_data['Area_per_Bedroom'] = housing_data['Area'] / housing_data['No. of Bedrooms']
  # Tạo các biến tương tác
  housing_data['Bedrooms_Gymnasium'] = housing_data['No. of Bedrooms'] * housing_data['Gymnasium']
  # Biến đổi log
  if train_data:
    housing_data['Log_Price'] = housing_data['Price'].apply(lambda x: np.log(x) if x > 0 else 0)
  # Biến đổi root
  housing_data['Root_Area'] = housing_data['Area'].apply(lambda x: np.sqrt(x))

  # Kết hợp các biến số học khác: MaintenanceStaff và CarParking
  housing_data['Staff_per_CarParking'] = housing_data['MaintenanceStaff'] / (housing_data['CarParking'] + 1)
  # Kết hợp các biến số học khác: SwimmingPool và Gymnasium
  housing_data['Pool_Gym'] = housing_data['SwimmingPool'] + housing_data['Gymnasium']
  # Kết hợp các biến số học khác: PowerBackup và 24X7Security
  housing_data['Power_Security'] = housing_data['PowerBackup'] + housing_data['24X7Security']
  # Tạo các biến tương tác khác: SportsFacility và ClubHouse
  housing_data['Sports_Club'] = housing_data['SportsFacility'] * housing_data['ClubHouse']
  # Tạo các biến tương tác khác: Intercom và VaastuCompliant
  housing_data['Intercom_Vaastu'] = housing_data['Intercom'] * housing_data['VaastuCompliant']

  # Kết hợp các biến số học khác: ATM và CarParking
  housing_data['ATM_per_CarParking'] = housing_data['ATM'] / (housing_data['CarParking'] + 1)
  # Kết hợp các biến số học khác: SportsFacility và SwimmingPool
  housing_data['Sports_Pool'] = housing_data['SportsFacility'] + housing_data['SwimmingPool']
  # Tạo các biến tương tác khác: ClubHouse và Gymnasium
  housing_data['Club_Gym'] = housing_data['ClubHouse'] * housing_data['Gymnasium']

  return housing_data

In [7]:
data_test = pd.read_csv('test.csv')

print(data_test.shape) # thiếu cột price
print(housing_data.shape) # thiếu cột resale do đã xóa

(6593, 40)
(26370, 38)


In [8]:
data_test = data_test.drop(columns=["Resale"]) # xóa resale ở dataset
data_test = data_test.drop(columns=["Location"]) # xóa location ở dataset
data_test = data_test.drop(columns=["id"]) # xóa location ở dataset
print(data_test.shape)

(6593, 37)


In [9]:
# mix feature
data_test = mix_feature(data_test, False)
housing_data = mix_feature(housing_data, True)
housing_data = housing_data.drop(columns=['Price']) # sử dụng log_price
print(housing_data.shape)
print(data_test.shape)

(26370, 49)
(6593, 48)


In [10]:
X = housing_data.drop(columns=['Log_Price'])  # Bỏ cột mục tiêu là log_price
y = housing_data['Log_Price']  # Lấy cột mục tiêu là log_price
print(X.shape)
print(data_test.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(26370, 48)
(6593, 48)


In [11]:
# data processing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numeric_columns = X.select_dtypes(include=[np.number]).columns.tolist()

encoder = OneHotEncoder(drop='first', sparse=False)
encoded_categorical_X = encoder.fit_transform(X[categorical_columns])
encoded_categorical_data_test = encoder.transform(data_test[categorical_columns])

encoded_categorical_df_train = pd.DataFrame(encoded_categorical_X, columns=encoder.get_feature_names_out(categorical_columns))
encoded_categorical_df_test = pd.DataFrame(encoded_categorical_data_test, columns=encoder.get_feature_names_out(categorical_columns))

data_train_encoded = pd.concat([X[numeric_columns], encoded_categorical_df_train], axis=1)
data_test_encoded = pd.concat([data_test[numeric_columns], encoded_categorical_df_test], axis=1)

scaler = StandardScaler()
scaled_data_train = scaler.fit_transform(data_train_encoded)
scaled_data_test = scaler.transform(data_test_encoded)

data_train_scaled = pd.DataFrame(scaled_data_train, columns=data_train_encoded.columns)
data_test_scaled = pd.DataFrame(scaled_data_test, columns=data_test_encoded.columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Hàm lựa chọn biến quan trọng bằng Random Forest Feature Importance
def select_features_random_forest(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]
    return X_train.columns[indices], importances[indices]

In [14]:
# Lựa chọn biến quan trọng bằng Random Forest
important_features_rf, importances_rf = select_features_random_forest(X_train, y_train)

# Hiển thị kết quả Random Forest Feature Importance
rf_feature_importance = pd.DataFrame({'Feature': important_features_rf, 'Importance': importances_rf})
print(rf_feature_importance)

                 Feature  Importance
0                   Area    0.293275
1              Root_Area    0.290470
2       Area_per_Bedroom    0.237310
3       MaintenanceStaff    0.021181
4   Staff_per_CarParking    0.017701
5     Bedrooms_Gymnasium    0.013885
6     ATM_per_CarParking    0.013617
7          Gasconnection    0.010934
8          LiftAvailable    0.005390
9     Children'splayarea    0.004917
10              Intercom    0.004905
11       No. of Bedrooms    0.004556
12   RainWaterHarvesting    0.004504
13                   BED    0.004297
14            CarParking    0.004265
15        Power_Security    0.003938
16                  Wifi    0.003599
17     LandscapedGardens    0.003247
18          StaffQuarter    0.002942
19              Pool_Gym    0.002904
20       VaastuCompliant    0.002886
21           Sports_Pool    0.002873
22           IndoorGames    0.002640
23          JoggingTrack    0.002591
24             ClubHouse    0.002500
25      MultipurposeRoom    0.002430
2

In [15]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [17]:
# dùng các feature trên 0.002
num_rf_features = 48
important_features_rf_selected = important_features_rf[:num_rf_features]

X_train_selected = X_train[important_features_rf_selected]
X_test_selected = X_test[important_features_rf_selected]

In [18]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Định nghĩa các mô hình hồi quy
seed = 42
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(random_state=seed),
    'Lasso': Lasso(random_state=seed),
    'DecisionTree': DecisionTreeRegressor(random_state=seed, max_depth=5),
    'RandomForest': RandomForestRegressor(random_state=seed),
    'GradientBoosting': GradientBoostingRegressor(random_state=seed),
    'ExtraTrees': ExtraTreesRegressor(random_state=seed),
    'AdaBoost': AdaBoostRegressor(random_state=seed),
    'XGBRegressor': XGBRegressor(random_state=seed),
    'LGBMRegressor': LGBMRegressor(random_state=seed),
    'CatBoostRegressor': CatBoostRegressor(silent=True, random_state=seed)
}

# Huấn luyện và đánh giá các mô hình
results = []
for model_name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append({
        'Model': model_name,
        'MAE': mae,
        'MSE': mse,
        'R2 Score': r2
    })

# Chuyển đổi kết quả thành DataFrame để hiển thị
results_df = pd.DataFrame(results, columns=['Model', 'MAE', 'MSE', 'R2 Score'])
results_df


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004688 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 924
[LightGBM] [Info] Number of data points in the train set: 21096, number of used features: 48
[LightGBM] [Info] Start training from score 15.822052


,Model,MAE,MSE,R2 Score
0,LinearRegression,0.544247,0.535898,0.203361
1,Ridge,0.544219,0.535871,0.203401
2,Lasso,0.593337,0.596527,0.113234
3,DecisionTree,0.537378,0.525271,0.219160
4,RandomForest,0.511743,0.535272,0.204293
5,GradientBoosting,0.525757,0.511599,0.239484
6,ExtraTrees,0.508525,0.547636,0.185913
7,AdaBoost,0.636516,0.647291,0.037770
8,XGBRegressor,0.505795,0.502325,0.253269
9,LGBMRegressor,0.509312,0.495833,0.262920


In [19]:
# Sort the results DataFrame by F1 Score in descending order
sorted_results_df = results_df.sort_values(by='R2 Score', ascending=False)
sorted_results_df

,Model,MAE,MSE,R2 Score
9,LGBMRegressor,0.509312,0.495833,0.262920
10,CatBoostRegressor,0.503797,0.496287,0.262245
8,XGBRegressor,0.505795,0.502325,0.253269
5,GradientBoosting,0.525757,0.511599,0.239484
3,DecisionTree,0.537378,0.525271,0.219160
4,RandomForest,0.511743,0.535272,0.204293
1,Ridge,0.544219,0.535871,0.203401
0,LinearRegression,0.544247,0.535898,0.203361
6,ExtraTrees,0.508525,0.547636,0.185913
2,Lasso,0.593337,0.596527,0.113234


Chọn Model LGBM và CatBoost

# Submit

In [23]:
LGBMRegressor_model = LGBMRegressor(silent=True, random_state=42)
LGBMRegressor_model.fit(X_train_selected, y_train)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 924
[LightGBM] [Info] Number of data points in the train set: 21096, number of used features: 48
[LightGBM] [Info] Start training from score 15.822052


LGBMRegressor(random_state=42, silent=True)

In [26]:
data_test_selected = data_test[important_features_rf_selected]
y_test_pred = LGBMRegressor_model.predict(data_test_selected)
y_test_pred[:5]

[LightGBM] [Warning] Unknown parameter: silent


array([15.83521667, 16.14645558, 15.94943555, 15.72868773, 15.60707498])